In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
VAL_PROMPT = "a logo of flower store, A poppy-like flower, red, with a black stem and green leaves, is arranged horizontally, with an inscription below it., white background, black, rosybrown foreground, minimalism, modern"
OUTPUT_DIR="text2img-lora-logo"

DATASET_NAME = "logo-wizard/modern-logo-dataset"
PRETRAINED_MODEL = "runwayml/stable-diffusion-v1-5"

In [ ]:
!pip install -q accelerate>=0.16.0 torchvision datasets diffusers bitsandbytes

In [ ]:
!accelerate config default

In [ ]:
!git clone https://github.com/docty/diffuser-training.git

In [ ]:
#!git clone https://huggingface.co/Docty/{MODEL_ID}

In [ ]:
import os

In [ ]:
!accelerate launch {os.getcwd()}/diffuser-training/train_text_to_image_lora.py \
  --pretrained_model_name_or_path="$PRETRAINED_MODEL"  \
  --dataset_name="$DATASET_NAME" --caption_column="text" \
  --resolution=128 --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=10 --checkpointing_steps=500 \
  --learning_rate=1e-04 --lr_scheduler="constant" --lr_warmup_steps=0 \
  --seed=42 \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --use_8bit_adam \
  --output_dir="$OUTPUT_DIR" \
  --validation_prompt="$VAL_PROMPT" \
  --resume_from_checkpoint='latest' \
  --checkpoints_total_limit=1 \
  --push_to_hub

# Inference

In [ ]:
from huggingface_hub.repocard import RepoCard
from diffusers import StableDiffusionPipeline
import torch

MODEL_ID = OUTPUT_DIR
lora_model_id = f"Docty/{MODEL_ID}"
card = RepoCard.load(lora_model_id)
base_model_id = card.data.to_dict()["base_model"]

pipe = StableDiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
#pipe.load_lora_weights(lora_model_id)
pipe.unet.load_attn_procs(lora_model_id)

In [ ]:
prompt = "a logo of cafe restaurant bar with a circle with an ornament on the sides, tableware above and two leaves on top and bottom, lightcyan background, midnightblue, midnightblue foreground, minimalism, modern"

image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
image